In [127]:
import pandas as pd
import numpy as np
from itertools import chain

In [2]:
input_path = "E:/CS_Master_Degree_UIUC/CS598_DeepLearning_for_Health_Data/Project/paper290/MIMIC data/"
output_path = "E:/CS_Master_Degree_UIUC/CS598_DeepLearning_for_Health_Data/Project/paper290/Processed_data/"

## ADMISSIONS.csv
1. Explore data and check what columns it has.
2. Find out all patients that have at least 2 admissions (at least 2 unique "HADM_ID"). Per Appendix1 of paper,  
   the dataset contains 7,537 patients with 2 or more encounters that they used in their CLOUT model.
3. Some demographic invetistigaiton such as distribution of ethnicity among the above subset of patient

In [15]:
# Read original dataset and explore the columns
admissions = pd.read_csv(input_path + "ADMISSIONS.csv")
col_names = admissions.columns
print(col_names)


# Findout the patient IDs ("SUBJECT_ID") that have at least 2 encounters 
AC = admissions.groupby(['SUBJECT_ID']).HADM_ID.nunique()
subset_id = AC.loc[AC > 1].reset_index().SUBJECT_ID	

# Get the subset of original dataset
admissions_subset = admissions.loc[admissions.SUBJECT_ID.isin(subset_id)].sort_values(by=['SUBJECT_ID','ROW_ID'])
admissions_subset = admissions_subset[['SUBJECT_ID', 'HADM_ID','ETHNICITY','HOSPITAL_EXPIRE_FLAG']]

# Verify if we get the right number of patients
patient_number = len(admissions_subset.SUBJECT_ID.unique())
print("The number of patients that have at least two encounters is: ",patient_number)
print("The number of records in admissions_subset is: ", admissions_subset.shape[0])
admissions_subset.head()

Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME',
       'DEATHTIME', 'ADMISSION_TYPE', 'ADMISSION_LOCATION',
       'DISCHARGE_LOCATION', 'INSURANCE', 'LANGUAGE', 'RELIGION',
       'MARITAL_STATUS', 'ETHNICITY', 'EDREGTIME', 'EDOUTTIME', 'DIAGNOSIS',
       'HOSPITAL_EXPIRE_FLAG', 'HAS_CHARTEVENTS_DATA'],
      dtype='object')
The number of patients that have at least two encounters is:  7537
The number of records in admissions_subset is:  19993


,SUBJECT_ID,HADM_ID,ETHNICITY,HOSPITAL_EXPIRE_FLAG
224,17,194023,WHITE,0
225,17,161087,WHITE,0
229,21,109451,WHITE,0
230,21,111970,WHITE,1
1,23,152223,WHITE,0


In [4]:
### Explore the ethnicity groups in the dataset
patient_ethnicity = pd.DataFrame(admissions_subset.groupby('ETHNICITY').SUBJECT_ID.unique()).reset_index()
patient_ethnicity['ETHNICITY_COUNT'] = patient_ethnicity.apply(lambda x: len(x['SUBJECT_ID']), axis = 1)
print("All ethnicity groups in original dataset: ", patient_ethnicity.ETHNICITY.values)

# Simplify the groups per paper into five basic groups

E_group = ['WHITE','BLACK','ASIAN','HISPANIC']

def Simple_Ethnicity(req, E_group):
    if req ==  'PORTUGUESE':
        return('WHITE')
    for e in E_group:
        if e in req:
            return(e)
    return('OTHER')

patient_ethnicity['ETHNICITY'] = patient_ethnicity.apply(lambda x: Simple_Ethnicity(x['ETHNICITY'], E_group), axis = 1)

patient_ethnicity.head()

All ethnicity groups in original dataset:  ['AMERICAN INDIAN/ALASKA NATIVE'
 'AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE' 'ASIAN'
 'ASIAN - ASIAN INDIAN' 'ASIAN - CAMBODIAN' 'ASIAN - CHINESE'
 'ASIAN - FILIPINO' 'ASIAN - KOREAN' 'ASIAN - OTHER' 'ASIAN - THAI'
 'ASIAN - VIETNAMESE' 'BLACK/AFRICAN' 'BLACK/AFRICAN AMERICAN'
 'BLACK/CAPE VERDEAN' 'BLACK/HAITIAN' 'CARIBBEAN ISLAND'
 'HISPANIC OR LATINO' 'HISPANIC/LATINO - CENTRAL AMERICAN (OTHER)'
 'HISPANIC/LATINO - COLOMBIAN' 'HISPANIC/LATINO - CUBAN'
 'HISPANIC/LATINO - DOMINICAN' 'HISPANIC/LATINO - GUATEMALAN'
 'HISPANIC/LATINO - HONDURAN' 'HISPANIC/LATINO - MEXICAN'
 'HISPANIC/LATINO - PUERTO RICAN' 'HISPANIC/LATINO - SALVADORAN'
 'MIDDLE EASTERN' 'MULTI RACE ETHNICITY'
 'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER' 'OTHER'
 'PATIENT DECLINED TO ANSWER' 'PORTUGUESE' 'UNABLE TO OBTAIN'
 'UNKNOWN/NOT SPECIFIED' 'WHITE' 'WHITE - BRAZILIAN'
 'WHITE - EASTERN EUROPEAN' 'WHITE - OTHER EUROPEAN' 'WHITE - RUSSIAN']


,ETHNICITY,SUBJECT_ID,ETHNICITY_COUNT
0,OTHER,"[5782, 14667, 20082, 31755, 40000, 43501]",6
1,OTHER,[42135],1
2,ASIAN,"[94, 191, 203, 299, 605, 907, 986, 1286, 1292,...",163
3,ASIAN,"[2666, 5962, 11043, 28278, 28860, 30650, 31120...",11
4,ASIAN,"[4334, 11171, 40474, 77471]",4


In [5]:
# Summarize the ethnicity group distribution among the patients and compare to Table-4 of Appendix1 of the paper
Ethnicity_table = patient_ethnicity.groupby('ETHNICITY').agg({'ETHNICITY_COUNT':sum})
Ethnicity_table['ETHNICITY_PERCENTAGE'] = np.round(Ethnicity_table['ETHNICITY_COUNT']/Ethnicity_table['ETHNICITY_COUNT'].sum()*100,1)
Ethnicity_table

,ETHNICITY_COUNT,ETHNICITY_PERCENTAGE
ETHNICITY,,
ASIAN,235,3.0
BLACK,885,11.3
HISPANIC,307,3.9
OTHER,665,8.5
WHITE,5736,73.3


## ICD codes
1. datasets include: DIGANOSES_ICD.csv and its dictionary dataset D_DIAGNOSES_ICD.csv. PROCEDURES_ICD.csv and its dictionnary      dataset D_PROCEDURES_ICD.csv.

In [106]:
diag_icd = pd.read_csv(input_path + 'DIAGNOSES_ICD.csv').drop('ROW_ID', axis = 1)
diag_icd_dict = pd.read_csv(input_path + 'D_ICD_DIAGNOSES.csv').drop('ROW_ID', axis = 1)
prod_icd = pd.read_csv(input_path + 'PROCEDURES_ICD.csv').drop('ROW_ID', axis = 1)
prod_icd_dict = pd.read_csv(input_path + 'D_ICD_PROCEDURES.csv').drop('ROW_ID', axis = 1)

In [107]:
diag_icd = diag_icd.merge(diag_icd_dict, on = 'ICD9_CODE', how='left')
prod_icd = prod_icd.merge(prod_icd_dict, on = 'ICD9_CODE', how='left')

In [103]:
print(diag_icd.columns.values)
print(prod_icd.columns.values)

['SUBJECT_ID' 'HADM_ID' 'SEQ_NUM' 'ICD9_CODE' 'SHORT_TITLE' 'LONG_TITLE']


In [115]:
#Combine both types of ICDs
ICDs = pd.concat([diag_icd, prod_icd], axis = 0).loc[subset_id].sort_values(['SUBJECT_ID', 'HADM_ID'])
#Group ICD9_CODE of per encounter of each patient  
ICDs = ICDs.groupby(['SUBJECT_ID', 'HADM_ID']).ICD9_CODE.unique().reset_index()
ICDs.head()

,SUBJECT_ID,HADM_ID,ICD9_CODE
0,3,145834,[4254]
1,5,178980,"[V3000, V053, V290, 9955]"
2,6,107064,"[9972, 2753, V1582]"
3,9,150750,"[2765, 9672]"
4,12,112213,"[99811, 5137, 5212, 3893]"


In [116]:
# Merge the ICDs with admission_subset
admissions_subset2 = admissions_subset.merge(ICDs, on  = ['SUBJECT_ID','HADM_ID'], how='left')
admissions_subset2.head()

,SUBJECT_ID,HADM_ID,ETHNICITY,HOSPITAL_EXPIRE_FLAG,ICD9_CODE
0,17,194023,WHITE,0,"[45829, V1259, 3961]"
1,17,161087,WHITE,0,NaN
2,21,109451,WHITE,0,"[78551, 5781, 4592, 4271, 28521, V1046]"
3,21,111970,WHITE,1,"[00845, 99592, 4439, E8788]"
4,23,152223,WHITE,0,"[4111, 2724, 60000, 3899, 3615]"


In [129]:
# Count the number of ICD9 codes
codes = admissions_subset2.loc[~admissions_subset2.ICD9_CODE.isna()]
print('the number of ICD codes is: ',len(set(list(chain(*codes.ICD9_CODE)))))

the number of ICD codes is:  1295


## Medication

In [16]:
prescriptions = pd.read_csv(input_path + 'PRESCRIPTIONS.csv', low_memory=False).drop(['ROW_ID'],axis = 1)
print(prescriptions.columns.values)

prescriptions.head()

['SUBJECT_ID' 'HADM_ID' 'ICUSTAY_ID' 'STARTDATE' 'ENDDATE' 'DRUG_TYPE'
 'DRUG' 'DRUG_NAME_POE' 'DRUG_NAME_GENERIC' 'FORMULARY_DRUG_CD' 'GSN'
 'NDC' 'PROD_STRENGTH' 'DOSE_VAL_RX' 'DOSE_UNIT_RX' 'FORM_VAL_DISP'
 'FORM_UNIT_DISP' 'ROUTE']


,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
0,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Tacrolimus,Tacrolimus,Tacrolimus,TACR1,021796,469061711.0,1mg Capsule,2,mg,2,CAP,PO
1,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Warfarin,Warfarin,Warfarin,WARF5,006562,56017275.0,5mg Tablet,5,mg,1,TAB,PO
2,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Heparin Sodium,NaN,NaN,HEPAPREMIX,006522,338055002.0,"25,000 unit Premix Bag","25,000",UNIT,1,BAG,IV
3,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,BASE,D5W,NaN,NaN,HEPBASE,NaN,0.0,HEPARIN BASE,250,ml,250,ml,IV
4,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Furosemide,Furosemide,Furosemide,FURO20,008208,54829725.0,20mg Tablet,20,mg,1,TAB,PO


In [130]:
medications = prescriptions[['SUBJECT_ID', 'HADM_ID','DRUG']]
medications = medications.groupby(['SUBJECT_ID', 'HADM_ID']).DRUG.unique().reset_index()
medications.head()

,SUBJECT_ID,HADM_ID,DRUG
0,2,163353,"[NEO*IV*Gentamicin, Syringe (Neonatal) *D5W*, ..."
1,4,185777,"[Iso-Osmotic Dextrose, Insulin, Benzonatate, D..."
2,6,107064,"[Tacrolimus, Warfarin, Heparin Sodium, D5W, Fu..."
3,8,159514,"[Send 500mg Vial, NEO*IV*Gentamicin, NEO*IV*Am..."
4,9,150750,"[SW, Labetalol HCl, Potassium Chloride, D5W, N..."


In [132]:
admissions_subset3 = admissions_subset2.merge(medications, on = ['SUBJECT_ID','HADM_ID'], how = 'left')
meds = admissions_subset3.loc[~admissions_subset3.DRUG.isna()]
print('the number of medication codes is: ',len(set(list(chain(*meds.DRUG)))))

admissions_subset3.head()
# admissions_subset3.SUBJECT_ID.nunique() #Check if still have 7537 patients

the number of medication codes is:  3202


,SUBJECT_ID,HADM_ID,ETHNICITY,HOSPITAL_EXPIRE_FLAG,ICD9_CODE,DRUG
0,17,194023,WHITE,0,"[45829, V1259, 3961]","[Sucralfate, Ketorolac, LR, Morphine Sulfate, ..."
1,17,161087,WHITE,0,NaN,"[Glycopyrrolate, Neostigmine, Calcium Gluconat..."
2,21,109451,WHITE,0,"[78551, 5781, 4592, 4271, 28521, V1046]","[Vial, Calcium Gluconate, Normocarb, Sterile W..."
3,21,111970,WHITE,1,"[00845, 99592, 4439, E8788]","[NS (Mini Bag Plus), Levothyroxine Sodium, Mer..."
4,23,152223,WHITE,0,"[4111, 2724, 60000, 3899, 3615]","[Syringe, Phenylephrine HCl, Nitroglycerin, LR..."


## Lab

In [137]:
labevents = pd.read_csv(input_path + 'LABEVENTS.csv').drop('ROW_ID',axis = 1)
d_labitems = pd.read_csv(input_path + 'D_LABITEMS.csv').drop('ROW_ID',axis = 1)
print(labevents.columns.values)
print(d_labitems.columns.values)

['SUBJECT_ID' 'HADM_ID' 'ITEMID' 'CHARTTIME' 'VALUE' 'VALUENUM' 'VALUEUOM'
 'FLAG']
['ITEMID' 'LABEL' 'FLUID' 'CATEGORY' 'LOINC_CODE']


In [140]:
labevents = labevents.merge(d_labitems, on = 'ITEMID', how = 'inner')[['SUBJECT_ID','HADM_ID','ITEMID','LABEL']]
labevents.head()

,SUBJECT_ID,HADM_ID,ITEMID,LABEL
0,3,NaN,50820,pH
1,3,NaN,50820,pH
2,3,NaN,50820,pH
3,3,145834.0,50820,pH
4,3,145834.0,50820,pH


In [141]:
labevents = labevents.groupby(['SUBJECT_ID', 'HADM_ID']).LABEL.unique().reset_index()
labevents.head()

,SUBJECT_ID,HADM_ID,LABEL
0,2,163353.0,"[Hematocrit, Anisocytosis, Basophils, Eosinoph..."
1,3,145834.0,"[pH, SPECIMEN TYPE, Base Excess, Calculated To..."
2,4,185777.0,"[pH, SPECIMEN TYPE, Base Excess, Calculated To..."
3,5,178980.0,"[Hematocrit, Basophils, Eosinophils, Hemoglobi..."
4,6,107064.0,"[pH, SPECIMEN TYPE, Base Excess, Calculated To..."


In [143]:
admissions_subset4 = admissions_subset3.merge(labevents, on = ['SUBJECT_ID','HADM_ID'], how = 'left')
labs = admissions_subset4.loc[~admissions_subset4.LABEL.isna()]
print('the number of lab item is: ',len(set(list(chain(*labs.LABEL)))))

admissions_subset4.head()

the number of lab item is:  545


,SUBJECT_ID,HADM_ID,ETHNICITY,HOSPITAL_EXPIRE_FLAG,ICD9_CODE,DRUG,LABEL
0,17,194023,WHITE,0,"[45829, V1259, 3961]","[Sucralfate, Ketorolac, LR, Morphine Sulfate, ...","[pH, SPECIMEN TYPE, Base Excess, Calculated To..."
1,17,161087,WHITE,0,NaN,"[Glycopyrrolate, Neostigmine, Calcium Gluconat...","[pH, SPECIMEN TYPE, Base Excess, Calculated To..."
2,21,109451,WHITE,0,"[78551, 5781, 4592, 4271, 28521, V1046]","[Vial, Calcium Gluconate, Normocarb, Sterile W...","[pH, SPECIMEN TYPE, Base Excess, Calculated To..."
3,21,111970,WHITE,1,"[00845, 99592, 4439, E8788]","[NS (Mini Bag Plus), Levothyroxine Sodium, Mer...","[pH, SPECIMEN TYPE, Base Excess, Calculated To..."
4,23,152223,WHITE,0,"[4111, 2724, 60000, 3899, 3615]","[Syringe, Phenylephrine HCl, Nitroglycerin, LR...","[pH, SPECIMEN TYPE, Base Excess, Calculated To..."
